In [1]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [2]:
import pandas as pd
from selenium import webdriver 
from bs4 import BeautifulSoup
import time
import re
import string, urllib2

In [3]:
from datetime import date, timedelta, datetime
import os.path
import tldextract
from collections import Counter

In [4]:
t0 =  date.today()

In [5]:
def get_columns():
    return ('dt', 'source', 'domain', 'url', 'title', 'summary', 'image', 'brief', 'body', 'raw_page', 'from') 
def get_empty_columns():
    return ('', '', '', '', '', '', '', '', '', '', '') 



In [6]:
def get_columns2():
    return ('url', 'domain', 'title', 'key_words', 'summary',  'segment', 'content', 'content_short',  'target') 
def get_empty_columns2():
    return ('', '', '', '', '', '', '', '', '') 

In [7]:
#complete
import glob
all_files = glob.glob("../data/complete/" + "/*.csv")

init = 0
#print all_files
for file_ in all_files:
    print file_
    with open(file_) as data_file:  
        df = pd.read_csv(data_file)
        if init == 0:
            joined = df
            init = 1
        else:
            joined = joined.append(df)

In [8]:
joined.info()

In [9]:
#News
import glob
all_files = glob.glob("../scrape/data/news/" + "/*.csv")

init = 0
#print all_files
for file_ in all_files:
    print file_
    with open(file_) as data_file:  
        df = pd.read_csv(data_file)
        joined = joined.append(df)

In [10]:
joined.info()

In [11]:
#general
import glob
all_files = glob.glob("../scrape/data/general/" + "/*.csv")

init = 0
#print all_files
for file_ in all_files:
    print file_
    with open(file_) as data_file:  
        df = pd.read_csv(data_file)
        joined = joined.append(df)

In [12]:
joined.info()

In [13]:
#CSE
import glob
all_files = glob.glob("../scrape/data/google/cse/" + "/*.csv")

#print all_files
for file_ in all_files:
    print file_
    with open(file_) as data_file:  
        df = pd.read_csv(data_file)
        joined = joined.append(df)

In [14]:
joined.info()

In [15]:
no_body_df = joined[joined['raw_page'].isnull()]

In [16]:
body_df = joined.dropna(subset =['raw_page']) 

In [17]:
#load body
import glob
all_files = glob.glob("./temp/" + "/*.csv")

init = 0
#print all_files
for file_ in all_files:
    print file_
    with open(file_) as data_file: 
        df = pd.read_csv(data_file)
        if init == 0:
            tmp_df = df
            init = 1
        else:
            tmp_df = tmp_df.append(df)

In [28]:
tmp_df.to_csv('temp.csv', encoding='utf-8', index=False)

In [31]:
tmp_df = pd.read_csv('temp.csv')

In [20]:
tmp_df.info()

In [21]:
tmp_df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,NaN,NaN,NaN,http://ah.anhuinews.com/system/2016/11/09/0075...,NaN,NaN,NaN,NaN,NaN,<html><head>\n\n<title>绩溪举办中国徽菜美食文化旅游节</title>...,NaN
1,NaN,NaN,NaN,http://epaper.anhuinews.com/html/ahsb/20160725...,NaN,NaN,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
2,NaN,NaN,NaN,http://ah.anhuinews.com/system/2016/11/09/0075...,NaN,NaN,NaN,NaN,NaN,<html><head>\n\n<title>绩溪举办中国徽菜美食文化旅游节</title>...,NaN
3,NaN,NaN,NaN,http://epaper.anhuinews.com/html/ahsb/20160725...,NaN,NaN,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
4,NaN,NaN,NaN,http://ah.anhuinews.com/system/2016/08/22/0074...,NaN,NaN,NaN,NaN,NaN,<html><head>\n\n<title>王信出席“文化中国之旅”安徽行启动仪式</ti...,NaN


In [22]:
def html_get_content(html, cut = 0):
    soup = BeautifulSoup(html.decode('utf-8'))
    [script.extract() for script in soup.findAll('script')]
    [style.extract() for style in soup.findAll('style')]
    soup.prettify()
    reg1 = re.compile("<[^>]*>")
    #content = soup
    content = reg1.sub('',soup.prettify())
    s = content.split('\n')
    ret = [a for a in s if len(a) > cut]
    return '\n'.join(ret)

In [23]:
new_df = pd.DataFrame(columns = get_columns2())

In [24]:
idx = 0

In [156]:
new_df.head()

,url,domain,title,key_words,summary,segment,content,content_short,target


In [132]:
for i in range(0, len(tmp_df)):
    url = tmp_df.loc[i]['url']
    print url
    #raw = tmp_df.loc[i]['raw_page']

In [25]:
from bosonnlp import BosonNLP

In [26]:
nlp = BosonNLP('9UKerw8k.13445.iKi57Vd-7oP3')
#nlp = BosonNLP('M6WKd8MT.13569.peZIQDEGBh9e')

In [58]:
for i in range(4003, len(tmp_df)):
    url = tmp_df.loc[i]['url']
    print i, url
    raw = tmp_df.loc[i]['raw_page']
    
    doc_short = html_get_content(raw, 60)
    doc = html_get_content(raw, 0)
    
    df_0 = joined[joined['url'] == url]
    if len(df_0) != 0 and len(doc_short) != 0:
        title = df_0.iloc[0]['title']
        
        tt20 = nlp.tag(doc_short)
        #time.sleep(5)
        tt2 = ' '.join(['%s' % it for it in tt20[0]['word']])

        res = nlp.extract_keywords(doc_short, top_k = 10)
        keys, values = zip(*res)
        if i == 686 or i == 1853 or i == 1882 or i == 1895:
            st = doc_short
        elif i == 1914 or i == 1939 or i == 1956 or i == 1977 or i == 2165:
            st = doc_short
        elif i == 2190 or i == 2222 or i == 3491:
            st = doc_short
        else:
            st = nlp.summary(title, doc_short)
        new_df.loc[idx] = get_empty_columns2()
        new_df.loc[idx]['url'] = url
        new_df.loc[idx]['title'] = title
        new_df.loc[idx]['key_words'] = values
        new_df.loc[idx]['summary'] = st
        new_df.loc[idx]['segment'] = tt2
        new_df.loc[idx]['content'] = doc
        new_df.loc[idx]['content_short'] = doc_short
        new_df.loc[idx]['target'] = 1
        idx += 1
    else:
        print 'cannot find it.........'

In [66]:
i = 1895
raw = tmp_df.loc[i]['raw_page']
doc_short = html_get_content(raw, 60)
#st = nlp.summary(title, doc_short)
new_df.loc[i]['summary'] = doc_short 

In [54]:
print len(doc_short), len(res)

In [57]:
print doc_short

In [53]:
new_df.info(), len(res)

(None, 0)

In [60]:
new_df.head()

,url,domain,title,key_words,summary,segment,content,content_short,target
0,http://ah.anhuinews.com/system/2016/11/09/0075...,,绩溪举办中国徽菜美食文化旅游节,"(旅游节, 徽菜, 文化, 美食, 绩溪, 张冬云, 品味, 严琛, 王为明, 出席)",据宣城新闻网报道，11月7日晚，中国绩溪徽菜美食文化旅游节隆重开幕。\n ...,据 宣城 新闻网 报道 ， 11月 7日 晚 ， 中国 绩溪徽菜 美食 文化 旅游节 隆重 ...,\n \n 绩溪举办中国徽菜美食文化旅游节\n \n \n \n \n \...,据宣城新闻网报道，11月7日晚，中国绩溪徽菜美食文化旅游节隆重开幕。省政协副主席...,1
1,http://epaper.anhuinews.com/html/ahsb/20160725...,,创文旅新格局传中华徽文化,"(万达, 合肥, 乐园, 欢乐, 城门, 业态, 游客, 华东, 旅游, 购票者)",集聚欢乐的人群，传递欢乐的业态，点燃欢乐的灯光，开售欢乐的门票。\n ...,集聚 欢乐 的 人群 ， 传递 欢乐 的 业态 ， 点燃 欢乐 的 灯光 ， 开 售 欢乐 ...,\n \n 创文旅新格局传中华徽文化 安徽商报\n \n \n \n \n ...,集聚欢乐的人群，传递欢乐的业态，点燃欢乐的灯光，开售欢乐的门票。仲夏夜的...,1
2,http://ah.anhuinews.com/system/2016/11/09/0075...,,绩溪举办中国徽菜美食文化旅游节,"(旅游节, 徽菜, 文化, 美食, 绩溪, 张冬云, 品味, 严琛, 王为明, 出席)",据宣城新闻网报道，11月7日晚，中国绩溪徽菜美食文化旅游节隆重开幕。\n ...,据 宣城 新闻网 报道 ， 11月 7日 晚 ， 中国 绩溪徽菜 美食 文化 旅游节 隆重 ...,\n \n 绩溪举办中国徽菜美食文化旅游节\n \n \n \n \n \...,据宣城新闻网报道，11月7日晚，中国绩溪徽菜美食文化旅游节隆重开幕。省政协副主席...,1
3,http://epaper.anhuinews.com/html/ahsb/20160725...,,创文旅新格局传中华徽文化,"(万达, 合肥, 乐园, 欢乐, 城门, 业态, 游客, 华东, 旅游, 购票者)",集聚欢乐的人群，传递欢乐的业态，点燃欢乐的灯光，开售欢乐的门票。\n ...,集聚 欢乐 的 人群 ， 传递 欢乐 的 业态 ， 点燃 欢乐 的 灯光 ， 开 售 欢乐 ...,\n \n 创文旅新格局传中华徽文化 安徽商报\n \n \n \n \n ...,集聚欢乐的人群，传递欢乐的业态，点燃欢乐的灯光，开售欢乐的门票。仲夏夜的...,1
4,http://ah.anhuinews.com/system/2016/08/22/0074...,,王信出席“文化中国之旅”安徽行启动仪式,"(文化, 侨办, 政府和市, 徽, 黄山市, 代表团, 侨团, 徽州, 王信, 徽墨)",8月20日，由国侨办宣传司主办、安徽省侨办承办的第21期“文化中国之旅”安徽行于...,8月 20日 ， 由 国侨办 宣传司 主办 、 安徽省 侨办 承办 的 第21 期 “ 文化...,\n \n 王信出席“文化中国之旅”安徽行启动仪式\n \n \n \n \...,8月20日，由国侨办宣传司主办、安徽省侨办承办的第21期“文化中国之旅”安徽行于...,1


In [71]:
new_df.to_csv('../data/segments/docs/bosonNLP/non-solr-bosonNLP-segment-4010.csv', encoding='utf-8', index=False)

In [69]:
new_df = pd.read_csv('../got_body-1.csv')

In [70]:
new_df.info()

In [131]:
no_body_df.info()

In [142]:
print raw

In [149]:
body_df['domain']  = body_df.url.map(lambda x:tldextract.extract(x).domain + str('.') + tldextract.extract(x).suffix)

In [151]:
count = Counter(body_df['domain'])

In [152]:
print count

In [ ]:
df0 = body_df[body_df['']]

In [81]:
body_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8925 entries, 0 to 104
Data columns (total 11 columns):
dt          8869 non-null object
source      8925 non-null object
domain      1125 non-null object
url         8925 non-null object
title       8925 non-null object
summary     0 non-null float64
image       0 non-null object
brief       3365 non-null object
body        6515 non-null object
raw_page    8925 non-null object
from        205 non-null object
dtypes: float64(1), object(10)
memory usage: 836.7+ KB


In [82]:
body_df.to_csv('temp/content.csv', encoding='utf-8', index=False)

In [17]:
print body_df.shape, no_body_df.shape

(8925, 11) (5720, 11)


In [18]:
no_body_df.to_csv('temp0.csv', encoding='utf-8', index=False)
no_body_df = pd.read_csv('temp0.csv')

In [19]:
no_body_df['domain']  = no_body_df.url.map(lambda x:tldextract.extract(x).domain + str('.') + tldextract.extract(x).suffix)

In [20]:
no_body_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5720 entries, 0 to 5719
Data columns (total 11 columns):
dt          4585 non-null object
source      5707 non-null object
domain      5720 non-null object
url         5720 non-null object
title       5717 non-null object
summary     0 non-null float64
image       1625 non-null object
brief       5538 non-null object
body        0 non-null float64
raw_page    0 non-null float64
from        2049 non-null object
dtypes: float64(3), object(8)
memory usage: 536.2+ KB


In [21]:
count = Counter(no_body_df['domain'])

In [66]:
print count.most_common(50)

[('anhuinews.com', 941), ('sina.com.cn', 842), ('chinanews.com', 465), ('360doc.com', 462), ('xinhuanet.com', 303), ('517huizhou.com', 294), ('sohu.com', 158), ('chinanews.com.cn', 148), ('163.com', 138), ('people.com.cn', 135), ('ifeng.com', 133), ('hf365.com', 126), ('xinhs.cn', 124), ('hzwh.com', 117), ('99huizhou.com', 97), ('newshs.com', 80), ('365jia.cn', 67), ('china.com.cn', 53), ('qq.com', 49), ('epochtimes.com', 44), ('kaixian.tv', 38), ('ahwang.cn', 32), ('eastday.com', 29), ('chinaqw.com', 28), ('ah.gov.cn', 26), ('gmw.cn', 24), ('chinadaily.com.cn', 23), ('zjol.com.cn', 19), ('china.com', 18), ('hfhouse.com', 17), ('sina.com.tw', 17), ('ce.cn', 16), ('radiovaticana.va', 13), ('cri.cn', 13), ('huaxia.com', 12), ('xinmin.cn', 11), ('hangzhou.com.cn', 11), ('huanqiu.com', 11), ('dwnews.com', 10), ('ahshuhua.net', 10), ('wehefei.com', 10), ('house365.com', 10), ('ntdtv.com', 9), ('cntv.cn', 9), ('srxww.com', 9), ('sciencenet.cn', 9), ('winshang.com', 9), ('cnr.cn', 8), ('hefei

def urllib_get_page_with_retry(url):
    stuff=None
    max_attempts=5
    #print idx, url
    r=0
    while stuff is None and r<max_attempts:
        try:
            response = urllib2.urlopen(url, timeout=2000)
            stuff = response.read()
        except urllib2.URLError:
            r=r+1
            print "Re-trying, attempt -- ",r
            time.sleep(5)
            pass
    return stuff


In [300]:
k = 17
url0 = count.most_common(50)[k][0]

In [301]:
temp_df = no_body_df[no_body_df['domain'] == url0]

In [302]:
temp_df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
12,2016-11-29,中国网福建频道,china.com.cn,http://fj.china.com.cn/p/251971.html,行走徽杭古道 体验徽州文化,NaN,NaN,"徽杭古道始称逍遥岩古道,初建于唐代,至今已有一千多年历史,与丝绸之路、茶马古道一起被称为“中...",NaN,NaN,NaN
62,2016-06-03,中国网,china.com.cn,http://cul.china.com.cn/2016-06/03/content_881...,古民居保护的黄山“模式”:守住徽文化“筋骨肉”,NaN,NaN,"黄山市是徽文化的发祥地、皖南民居集聚区,也是皖南国际文化旅游示范区和徽州文化生态保护...",NaN,NaN,NaN
63,2016-06-01,东海资讯,china.com.cn,http://jiangsu.china.com.cn/html/2016/ahnews_0...,"街口镇传承徽文化,徽剧进校园",NaN,NaN,"徽剧为国粹京剧的起源奠定了坚实的基础,是徽文化的重要组成部分,而街源地区是徽剧的发祥地。街口...",NaN,NaN,NaN
95,2016-01-29,中国视窗,china.com.cn,http://zgsc.china.com.cn/jianghuai/whys/2016-0...,黄山推动徽文化保护传承 继续推进徽州古建筑保护工程,NaN,NaN,"从日前召开的黄山市六届人大六次会议上了解到,“十三五”期间,黄山市在继续推进徽州古建筑保护利...",NaN,NaN,NaN
158,2015-10-15,中国视窗,china.com.cn,http://zgsc.china.com.cn/jixi/tswh/2015-10-15/...,绩溪——沉甸甸的“徽文化博物馆”,NaN,NaN,"正所谓“无徽无成镇,无绩不成街”,作为徽州六县的绩溪是徽州文化的发源地之一。高铁绩溪北站,是...",NaN,NaN,NaN


In [303]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 12 to 2812
Data columns (total 11 columns):
dt          45 non-null object
source      53 non-null object
domain      53 non-null object
url         53 non-null object
title       53 non-null object
summary     0 non-null float64
image       30 non-null object
brief       53 non-null object
body        0 non-null float64
raw_page    0 non-null float64
from        37 non-null object
dtypes: float64(3), object(8)
memory usage: 5.0+ KB


In [304]:
temp_df.to_csv('temp.csv', encoding='utf-8', index=False)

In [305]:
#no_body_df.to_csv('temp.csv', encoding='utf-8', index=False)
df = pd.read_csv('temp.csv')

In [306]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 11 columns):
dt          45 non-null object
source      53 non-null object
domain      53 non-null object
url         53 non-null object
title       53 non-null object
summary     0 non-null float64
image       30 non-null object
brief       53 non-null object
body        0 non-null float64
raw_page    0 non-null float64
from        37 non-null object
dtypes: float64(3), object(8)
memory usage: 5.0+ KB


In [307]:
browser = webdriver.Firefox() 

In [295]:
start = 0
idx = 0

In [255]:
print df.loc[102]['url']

http://www.hzwh.com/goto.asp?topicid=231


In [308]:
new_df = pd.DataFrame(columns = get_columns())

In [309]:
for i in range(0, len(df)):
    
    url = df.loc[i]['url']
    print i, url
    
    stuff = None
    #urllib_get_page_with_retry(url)
    if stuff == None:
            browser.get(url)
            time.sleep(5)
            stuff = browser.page_source 
    #urllib_get_page_with_retry(url)
    #time.sleep(5)
    
    new_df.loc[idx] = get_empty_columns()        
    new_df.loc[idx]['url'] = url
    new_df.loc[idx]['raw_page'] = stuff
    idx += 1
    

0 http://fj.china.com.cn/p/251971.html


TimeoutException: Message: Error loading page, timed out (onDOMContentLoaded)


In [158]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 137
Data columns (total 11 columns):
dt          138 non-null object
source      138 non-null object
domain      138 non-null object
url         138 non-null object
title       138 non-null object
summary     138 non-null object
image       138 non-null object
brief       138 non-null object
body        138 non-null object
raw_page    138 non-null object
from        138 non-null object
dtypes: object(11)
memory usage: 12.9+ KB


In [232]:
new_df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,,,,http://epaper.hf365.com/sqb/gxxw/html/2016-07/...,,,,,,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
1,,,,http://epaper.hf365.com/jhcb/html/2016-04/10/c...,,,,,,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
2,,,,http://epaper.hf365.com/hfrb/html/2015-05/05/c...,,,,,,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
3,,,,http://epaper.hf365.com/hfrb/html/2015-04/03/c...,,,,,,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
4,,,,http://epaper.hf365.com/hfwb/html/2015-03/16/c...,,,,,,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",


In [90]:
print stuff

In [287]:
new_df.to_csv('temp/temp13.csv', encoding='utf-8', index=False)

In [35]:
m = new_df.loc[1]['raw_page']
soup = BeautifulSoup(m)
print soup.text


创文旅新格局传中华徽文化 安徽商报







分享到一键分享QQ空间新浪微博百度云收藏人人网腾讯微博百度相册开心网腾讯朋友百度贴吧豆瓣网搜狐微博百度新首页QQ好友和讯微博更多...百度分享

	
		
			









			09 民生新闻·资讯

var str=window.location.href;
if(str.indexOf("ahrb")>0||str.indexOf("wjzk")>0||str.indexOf("jrng")>0||str.indexOf("ngnews")>0||str.indexOf("dzxw")>0||str.indexOf("sszk")>0||str.indexOf("tdjs")>0||str.indexOf("crec4")>0||str.indexOf("zyzk")>0||str.indexOf("zyxw")>0){document.write('<div style="margin-top:10px;text-align:center;display:none">')}else{document.write('<div style="margin-top:10px;text-align:center;">')}
//安徽日报不显示广告



/*数字报内容*/
var cpro_id = "u1455815";



document.write('</div>');
		
		
			2016年07月25日 星期一国内统一刊号:CN34-0044
			
			
				安徽日报报业集团报系：
								
				
					请选择报纸安徽日报新安晚报安徽商报安徽日报农村版安徽法制报江淮时报世界报新闻世界徽商
				 日期 
							
				更多
			
			订阅安徽手机报　发送hah到10658000即可随身阅读每天早晚报　5元/月
			
				
					
						
						分享到：
						
						
						
						
						
						0
						
						
						
						
						document.getElementById("bdshell_js").src = "http://bdimg.share

/usr/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /usr/local/Cellar/python/2.7.10_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [68]:
print m

None


In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4496 entries, 0 to 4495
Data columns (total 11 columns):
dt          4496 non-null object
source      4496 non-null object
domain      4496 non-null object
url         4496 non-null object
title       4496 non-null object
summary     4496 non-null object
image       4496 non-null object
brief       4496 non-null object
body        4496 non-null object
raw_page    4496 non-null object
from        4496 non-null object
dtypes: object(11)
memory usage: 421.5+ KB


In [12]:
for i in range(0, 1):
    url_0 = news_url
    print url_0
    browser.get(url_0)
    time.sleep(5)

    soup = BeautifulSoup(browser.page_source)
    sub_text = soup.findAll('div', {'class':'listA_title'})

    for item in sub_text:
        t1 = item.find('div', {'class':'listA_Time'})
        #print 'www.hzwh.com/' + item.a['href']
        #print t1.text
        print item.a.text
        url = 'www.hzwh.com/' + item.a['href']
        
        not_existed = True
        #check if in cleaned set
        if t1 != None and len(df_check) != 0:
            t0 = abs(sum(df_check['url'].str.find(url))) 
            if t0 != len(df_check):
                not_existed = False

        #check if in dynamic set
        if t1 != None and len(df) != 0 and not_existed == True:
            t0 = abs(sum(df['url'].str.find(str(url)))) 
            if t0 != len(df):
                not_existed = False

        #print t0, len(df), t1

        if not_existed == True and t1 != None:   
            print "-------- add"
            df.loc[idx] = get_empty_columns()        
            df.loc[idx]['url'] = url
            df.loc[idx]['title'] = item.a.text
            df.loc[idx]['source'] = '徽州文化网'
            df.loc[idx]['dt'] = t1.text
            idx += 1

http://www.hzwh.com/toplistb.asp?id=229&page=0
传统徽州仁商义贾践行商君子之道的历史启示
《阅读徽州》第七编“徽州科技”
传统徽商的诚信为本和仁义向善
传统徽商的诚信为本和仁义向善
黟县古城改造重要的是务须保住文教根脉
推动徽州传统文化的普及宣传
徽学与当代学术 
“儒学与地域文化：徽学国际学术研讨会”综述
洪经纶墓葬地点的疏证
徽州乡镇志中所见明清民间救荒措施
徽州文化全书
论文化生产力与中部地区的经济发展
徽州书院的讲会
明代海上徽帮与宁波海上丝绸之路
徽州昌溪周氏家族与宁波近代商业发展史
徽商历史的文化价值批判 
徽商
徽商文化及特点
婺源徽剧音乐研究
古徽州和谐乡村社会建设的经验


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /usr/local/Cellar/python/2.7.10_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [13]:
print len(df)

0


In [12]:
df.head(50)

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2004-03-03,徽州文化网,,www.hzwh.com/article.asp?id=533,旧时徽州税票名称种种,,,,,,
1,2004-03-03,徽州文化网,,www.hzwh.com/article.asp?id=532,小小大阜村 爷孙宰相俩,,,,,,
2,2004-03-09,徽州文化网,,www.hzwh.com/article.asp?id=530,李良春与《惯痴文集》,,,,,,
3,2004-03-22,徽州文化网,,www.hzwh.com/article.asp?id=528,古老的石林,,,,,,
4,2004-03-31,徽州文化网,,www.hzwh.com/article.asp?id=527,洪钧行书联,,,,,,
5,2004-04-02,徽州文化网,,www.hzwh.com/article.asp?id=525,万安“中国罗盘之乡”,,,,,,
6,2004-04-11,徽州文化网,,www.hzwh.com/article.asp?id=523,吕仙宫,,,,,,
7,2004-04-12,徽州文化网,,www.hzwh.com/article.asp?id=521,溪 头 探 古,,,,,,
8,2004-04-15,徽州文化网,,www.hzwh.com/article.asp?id=520,五绝《小桃源》中否李白所作？,,,,,,
9,2004-04-15,徽州文化网,,www.hzwh.com/article.asp?id=518,解读戴震诗一首,,,,,,


In [28]:
print len(df)

0


In [163]:
def url_get(url):
    browser.get(url)
    print url
    time.sleep(5)
    return browser.page_source

In [164]:
df['dt'] = df_check['dt']
df['source'] = df_check['source']
df['url'] = df_check['url']
df['title'] = df_check['title']
df['raw_page'] = df_check.url.map(lambda x: url_get('http://' + x) )  

http://www.hzwh.com/article.asp?id=2538
http://www.hzwh.com/article.asp?id=2537
http://www.hzwh.com/article.asp?id=2536
http://www.hzwh.com/article.asp?id=2535
http://www.hzwh.com/article.asp?id=2534
http://www.hzwh.com/article.asp?id=2533
http://www.hzwh.com/article.asp?id=2099
http://www.hzwh.com/article.asp?id=2098
http://www.hzwh.com/article.asp?id=1848
http://www.hzwh.com/article.asp?id=1847
http://www.hzwh.com/article.asp?id=1846
http://www.hzwh.com/article.asp?id=1794
http://www.hzwh.com/article.asp?id=1695
http://www.hzwh.com/article.asp?id=1694
http://www.hzwh.com/article.asp?id=1693
http://www.hzwh.com/article.asp?id=1624
http://www.hzwh.com/article.asp?id=1623
http://www.hzwh.com/article.asp?id=1622
http://www.hzwh.com/article.asp?id=1455
http://www.hzwh.com/article.asp?id=1418
http://www.hzwh.com/article.asp?id=1417
http://www.hzwh.com/article.asp?id=1416
http://www.hzwh.com/article.asp?id=1415
http://www.hzwh.com/article.asp?id=1414
http://www.hzwh.com/article.asp?id=1347


TimeoutException: Message: Error loading page, timed out (onDOMContentLoaded)


In [165]:
df['url'] = 'http://' + df['url']

In [119]:
df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2017-2-9,故园徽州,,forum.php?mod=viewthread&tid=195025&extra=page...,【春节民俗活动预告】——中云镇桃溪坑头“抬汪公”民俗,,,,\n\n\n\n\n马上注册成为会员，享用更多功能，让你轻松玩转徽州\n您需要 登录 才可以...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
1,2017-1-1,故园徽州,,forum.php?mod=viewthread&tid=194254&extra=page...,2017.01.01的早上,,,,\n\n\n\n\n马上注册成为会员，享用更多功能，让你轻松玩转徽州\n您需要 登录 才可以...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
2,2016-11-23,故园徽州,,forum.php?mod=viewthread&tid=192264&extra=page...,上学去喽！,,,,\n\n\n\n\n马上注册成为会员，享用更多功能，让你轻松玩转徽州\n您需要 登录 才可以...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
3,2016-10-25,故园徽州,,forum.php?mod=viewthread&tid=189658&extra=page...,流淌的登源河（29）：老造反成了普通看客,,,,\n\n\n\n\n马上注册成为会员，享用更多功能，让你轻松玩转徽州\n您需要 登录 才可以...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",
4,2016-10-17,故园徽州,,forum.php?mod=viewthread&tid=189555&extra=page...,沙溪“淩”姓来由,,,,\n\n\n\n\n马上注册成为会员，享用更多功能，让你轻松玩转徽州\n您需要 登录 才可以...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",


In [152]:
characters_num = 40
def get_brief(txt):
    soup = BeautifulSoup(txt)
    sub_text = soup.findAll('div', {'class':'article_contents'})
    for item in sub_text:
        t20 = item.text

    idx = 2
    t30 = t20.split(u'\n')
    
    cnt = 0
    tt = ''
    idx = 0
    for k in range(0, len(t30)):
        if (len(t30[k]) > 40 and len(tt) < 40):
            idx = k
            tt = t30[k].strip()
    
    if len(tt) == 0:
        tt = t20.strip()
                
    #print idx

    t40 = tt[0:characters_num]
    t50 = t40.encode('utf-8') + '..'
    #print idx, t40
    return t50

In [158]:
df['brief'] = df.raw_page.map(lambda x: get_brief(x) )  

In [154]:
def get_body(txt):
    soup = BeautifulSoup(txt)
    sub_text = soup.findAll('div', {'class':'article_contents'})
    for item in sub_text:
        t20 = item.text

    return t20

In [159]:
df['body'] = df.raw_page.map(lambda x: get_body(x) )  

cnt = 0
for item in df['brief']:
    print cnt, item
    cnt += 1

cnt = 0
for item in df['body']:
    print cnt, item
    cnt += 1

soup = BeautifulSoup(df.loc[527]['raw_page'])
sub_text = soup.findAll('div', {'class':'article_contents'})
for item in sub_text:
    t20 = item.text
t30 = t20.split(u'\n')

tt = t20.strip()
print len(t30), tt[0:40], t20

In [161]:
df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2016-04-08,徽州文化网,NaN,www.hzwh.com/article.asp?id=2538,传统徽州仁商义贾践行商君子之道的历史启示,NaN,NaN,全国人大代表、民盟安徽省委副主委、安徽省文联副主席钱念孙先生在《光明日报》201..,传统徽州仁商义贾践行商君子之道的历史启示\n郗延红\n全国人大代表、民盟安徽省委副主委、安徽...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
1,2016-04-08,徽州文化网,NaN,www.hzwh.com/article.asp?id=2537,《阅读徽州》第七编“徽州科技”,NaN,NaN,长期以来，得益于灵山秀水自然环境的滋养、兴学重教社会风气的熏染、经世致用治学风格..,《阅读徽州》第七编“徽州科技”（市委市政府通过市社科联组织编写）\n中国科学技术学大出版社出...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
2,2016-04-08,徽州文化网,NaN,www.hzwh.com/article.asp?id=2536,传统徽商的诚信为本和仁义向善,NaN,NaN,徽州古楹联云“仁中取利真君子/义内求财大丈夫”。这是对传统徽商群体中那些仁商义贾..,传统徽商的诚信为本和仁义向善\n\n郗延红\n徽州古楹联云“仁中取利真君子/义内求财大丈夫”...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
3,2016-04-08,徽州文化网,NaN,www.hzwh.com/article.asp?id=2535,传统徽商的诚信为本和仁义向善,NaN,NaN,徽州古楹联云“仁中取利真君子/义内求财大丈夫”。这是对传统徽商群体中那些仁商义贾..,传统徽商的诚信为本和仁义向善\n \n郗延红\n徽州古楹联云“仁中取利真君子/义内求财大丈夫...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
4,2016-04-08,徽州文化网,NaN,www.hzwh.com/article.asp?id=2534,黟县古城改造重要的是务须保住文教根脉,NaN,NaN,黟县进行古城改造并申报国家级历史文化名城总体方案正在逐步完善，各项工作正在有条不..,黟县古城改造重要的是务须保住文教根脉\n 郗...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN


In [127]:
df.loc[24]['url']

'www.hzwh.com/article.asp?id=1347'

In [121]:
if(len(df) != 0):
    df.to_csv('../data/complete/hzwh_huixueyuanpu.csv', encoding='utf-8', index=False)

In [30]:
if(browser):
    browser.quit()

In [31]:
for item in df['title']:
    print item

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 11 columns):
dt          0 non-null object
source      0 non-null object
domain      0 non-null object
url         0 non-null object
title       0 non-null object
summary     0 non-null object
image       0 non-null object
brief       0 non-null object
body        0 non-null object
raw_page    0 non-null object
from        0 non-null object
dtypes: object(11)
memory usage: 0.0+ bytes


In [173]:
df2 = pd.read_csv('../data/complete/hzwh_huixueyuanpu.csv')

In [174]:
df2.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2016-04-08,徽州文化网,NaN,http://www.hzwh.com/article.asp?id=2538,传统徽州仁商义贾践行商君子之道的历史启示,NaN,NaN,全国人大代表、民盟安徽省委副主委、安徽省文联副主席钱念孙先生在《光明日报》201..,传统徽州仁商义贾践行商君子之道的历史启示\n郗延红\n全国人大代表、民盟安徽省委副主委、安徽...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
1,2016-04-08,徽州文化网,NaN,http://www.hzwh.com/article.asp?id=2537,《阅读徽州》第七编“徽州科技”,NaN,NaN,长期以来，得益于灵山秀水自然环境的滋养、兴学重教社会风气的熏染、经世致用治学风格..,《阅读徽州》第七编“徽州科技”（市委市政府通过市社科联组织编写）\n中国科学技术学大出版社出...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
2,2016-04-08,徽州文化网,NaN,http://www.hzwh.com/article.asp?id=2536,传统徽商的诚信为本和仁义向善,NaN,NaN,徽州古楹联云“仁中取利真君子/义内求财大丈夫”。这是对传统徽商群体中那些仁商义贾..,传统徽商的诚信为本和仁义向善\n\n郗延红\n徽州古楹联云“仁中取利真君子/义内求财大丈夫”...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
3,2016-04-08,徽州文化网,NaN,http://www.hzwh.com/article.asp?id=2535,传统徽商的诚信为本和仁义向善,NaN,NaN,徽州古楹联云“仁中取利真君子/义内求财大丈夫”。这是对传统徽商群体中那些仁商义贾..,传统徽商的诚信为本和仁义向善\n \n郗延红\n徽州古楹联云“仁中取利真君子/义内求财大丈夫...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN
4,2016-04-08,徽州文化网,NaN,http://www.hzwh.com/article.asp?id=2534,黟县古城改造重要的是务须保住文教根脉,NaN,NaN,黟县进行古城改造并申报国家级历史文化名城总体方案正在逐步完善，各项工作正在有条不..,黟县古城改造重要的是务须保住文教根脉\n 郗...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",NaN


In [83]:
t_df = pd.read_csv('temp/temp01.csv')

In [86]:
t_df.loc[0]['raw_page']

AttributeError: 'str' object has no attribute 'text'